# Exploring Hypothesis-The-Library Testing within cadCAD using the Minimal model
By: Matthew Barlin 

The longest test on the shortest model ever

Credit to: *Danilo Lessa Bernardineli* for creating the shortest model ever

In [1]:
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD import configs
import pandas as pd
import numpy as np

In [2]:
from hypothesis import event, given, strategies as st
import inspect


In [3]:
MONTE_CARLO_RUNS = 1
SIMULATION_TIMESTEPS = 3

params = {
    'parameter': [1],
    'pass_fail': [0,1]
}

genesis_states = {
    'something': 0,
    'something_tested' : 0
}

def p_random_number(params, 
                substep, 
                state_history, 
                prev_state):
    new_value = np.random.randn()
    return {'random_number': new_value}


def read_in_random(value):
    return value

# def test_read_in_random(value):
#     round(value)
#     assert value == round(value)

@given(value = st.integers())
def test_read_in_random(value):
    new_value = read_in_random(value)
    assert new_value == round(value)
    # return value

def s_something(params, 
                substep, 
                state_history, 
                prev_state, 
                policy_input):
    new_value = read_in_random(policy_input['random_number'])
    return ('something', new_value)

# @given(st.dictionaries(), st.dictionaries(),st.dictionaries(),st.dictionaries(),st.dictionaries())
def s_something_tested(params, 
                substep, 
                state_history, 
                prev_state, 
                policy_input):
    # new_value = test_read_in_random(policy_input['random_number'])
    
    if params['pass_fail'] == 0:
       print('Expecting test will pass')
    if params['pass_fail'] == 1:
       print('Expecting test will fail')
    
    @given(value = st.integers())
    def test_read_in_random(value):
        new_value = read_in_random(value)
        assert new_value == value + params['pass_fail']

    test_read_in_random()
    
    if params['pass_fail'] == 0:
       print('Test passed as expected')
    if params['pass_fail'] == 1:
       print('Test passed as expected')
    new_value = read_in_random(policy_input['random_number'])
    
    return ('something_tested', new_value)

partial_state_update_blocks = [
    {
        'policies': {
            'random_number': p_random_number
            
        },
        'variables': {
            'something': s_something,
            'something_tested': s_something_tested
        }
    }
]

sim_config = {
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': params
}

In [4]:
print(inspect.getsource(test_read_in_random))

@given(value = st.integers())
def test_read_in_random(value):
    new_value = read_in_random(value)
    assert new_value == round(value)
    # return value



In [5]:
test_read_in_random()

In [6]:
sim_params = config_sim(sim_config)

exp = Experiment()
exp.append_configs(
    sim_configs=sim_params,
    initial_state=genesis_states,
    partial_state_update_blocks=partial_state_update_blocks
)

exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)

simulation = Executor(exec_context=local_mode_ctx,
                      configs=configs)
raw_system_events, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (3, 2, 2, 2)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 1]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized
Expecting test will pass
Test passed as expected
Expecting test will pass
Test passed as expected
Expecting test will pass
Test passed as expected
Expecting test will fail
Falsifying example: test_read_in_random(
    value=0,
)


AssertionError: 

In [ ]:
df = pd.DataFrame(raw_system_events)

In [ ]:
df.head()

In [ ]:

df.set_index('timestep')['something'].plot()
df.set_index('timestep')['something_tested'].plot()
